In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import json
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, roc_auc_score

from divopt.chem import ebv2np, morgan_from_smiles_list
from divopt.evaluation.selection import reverse_taylor_butina
from divopt.utils import filterdf

In [1]:
def train_classifier(
    scoring_function_dir: str,
    train_fraction: float,
    distance_threshold: float,
    n_jobs=8,
    seed=0,
):
    """Train a random forest classifier to predict the activity of a molecule given its Morgan fingerprint.

    Args:
        scoring_function_dir (str): Directory where the scoring function will be saved.
        train_fraction (float): Fraction of the data that will be used for training.
        threshold (float): Threshold used to determine if a molecule is active or inactive.
        distance_threshold (float): Threshold used to determine if two molecules are similar.
        n_jobs (int, optional): Number of jobs to run in parallel. Defaults to 8.
        seed (int, optional): Random seed. Defaults to 0.
    """
    # load data
    np.random.seed(seed)
    data_fn = os.path.join(scoring_function_dir, "all.txt")
    df = pd.read_csv(data_fn)
    df.columns = ["smiles", "label"]

    # compute fingerprints
    smiles_list = df["smiles"].tolist()
    fp_list = morgan_from_smiles_list(smiles_list, radius=2, nbits=2048, n_jobs=n_jobs)

    # train/test split
    fp_list_np = [ebv2np(fp) for fp in fp_list]
    assert not any(x is None for x in fp_list_np)

    X = np.array([ebv2np(fp) for fp in fp_list])
    y = df["label"].to_numpy()
    random_idx = np.random.permutation(len(df))
    n_train_samples = int(len(df) * train_fraction)
    train_idx = random_idx[:n_train_samples]
    test_idx = random_idx[n_train_samples:]
    X_train = X[train_idx]
    X_test = X[test_idx]
    y_train = y[train_idx]
    y_test = y[test_idx]

    # add split information to dataframe
    split_array = np.array(["train"] * (len(df)))
    split_array[test_idx] = "test"
    df["Split"] = split_array
    df.to_csv(os.path.join(scoring_function_dir, "splits.csv"), index=False)

    # train classifier
    clf = RandomForestClassifier(n_jobs=8)
    clf.fit(X_train, y_train)

    y_score = clf.predict_proba(X_test)[:, 1]
    # record some stats
    stats = {}

    # Performance metrics of classifier
    stats["ROCAUC"] = roc_auc_score(y_test, y_score)
    stats["AP"] = average_precision_score(y_test, y_score)
    for threshold in np.arange(0, 1, 0.1):
        stats[f"F1@{threshold:.2f}"] = f1_score(y_test, y_score > threshold)
        stats[f"Prec@{threshold:.2f}"] = precision_score(y_test, y_score > threshold)
        stats[f"Rec@{threshold:.2f}"] = recall_score(y_test, y_score > threshold)

    # Active score stats
    y_score_actives_test = y_score[y_test == 1]
    y_score_actives_train = clf.predict_proba(X_train)[:, 1][y_train == 1]
    stats["Avg. active score (test)"] = (y_score_actives_test.mean(), y_score_actives_test.std())
    stats["Avg. active score (train)"] = (y_score_actives_train.mean(), y_score_actives_train.std())

    # Dataset stats
    stats["#Samples"] = len(df)
    train_actives = filterdf(df, {"Split": "train", "label": 1}).smiles
    actives = filterdf(df, {"label": 1}).smiles

    # Diverse actives
    stats["#Train actives"] = len(train_actives)
    stats[f"#Train actives@{distance_threshold:.2f}"] = len(
        reverse_taylor_butina(
            smiles=train_actives.to_list(), distance_threshold=distance_threshold, radius=2, nbits=2048
        )
    )
    stats[f"#Actives@{distance_threshold:.2f}"] = len(
        reverse_taylor_butina(smiles=actives.to_list(), distance_threshold=distance_threshold, radius=2, nbits=2048)
    )

    # save classifier
    clf_fn = os.path.join(scoring_function_dir, "classifier.pkl")
    with open(clf_fn, "wb") as f:
        pickle.dump(clf, f)

    # save stats
    stats_fn = os.path.join(scoring_function_dir, "stats.json")
    with open(stats_fn, "w") as f:
        json.dump(stats, f, indent=True)

## Train the models

In [27]:
targets = ["jnk3", "gsk3", "drd2"]

train_fraction = 0.75
score_threshold = 0.5
distance_threshold = 0.7
divopt_dir = os.environ["DIVOPTPATH"]
scoring_function_base = os.path.join(divopt_dir, "data/scoring_functions/")
scoring_function_dirs = [os.path.join(scoring_function_base, target) for target in targets]

In [ ]:
for scoring_function_dir in scoring_function_dirs:
    train_classifier(
        scoring_function_dir=scoring_function_dir,
        train_fraction=train_fraction,
        distance_threshold=distance_threshold,
    )

## Display some metrics

In [29]:
# Load stats from scoring functions and join them in dataframe
stats_list = []
for scoring_function_dir in scoring_function_dirs:
    stats_fn = os.path.join(scoring_function_dir, "stats.json")
    with open(stats_fn, "r") as f:
        stats = json.load(f)
    stats["Target"] = scoring_function_dir.split("/")[-1]
    stats_list.append(stats)

stats_df = pd.DataFrame(stats_list)
# put last column in first position
cols = stats_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
stats_df = stats_df[cols]

stats_df["Target"] = stats_df["Target"].apply(lambda x: x.upper())


# add column with formatted active scores (mean +- std)
stats_df["Avg. active score (test)"] = stats_df["Avg. active score (test)"].apply(
    lambda x: rf"${x[0]:.2f} \pm {x[1]:.2f}$"
)
stats_df["Avg. active score (train)"] = stats_df["Avg. active score (train)"].apply(
    lambda x: rf"${x[0]:.2f} \pm {x[1]:.2f}$"
)

# rename columns
stats_df = stats_df.rename(
    columns={
        "Avg. active score (train)": "Scores (train)",
        "Avg. active score (test)": "Scores (test)",
        "#Train actives": "#Act (train)",
        f"#Train actives@{distance_threshold}": f"#Act@{distance_threshold} (train)",
        f"#Actives@{distance_threshold}": f"#Act@{distance_threshold}",
    }
)  # type: ignore

table_columns = ["Target", "ROCAUC", "AP", "Prec@0.5", "Rec@0.5", "Prec@0.9", "Rec@0.9", "#Samples", "#Actives@0.70"]
df_latex = stats_df[table_columns]
df_latex

,Target,ROCAUC,AP,F1@0.0,Prec@0.00,Rec@0.00,F1@0.1,Prec@0.10,Rec@0.10,F1@0.2,...,Rec@0.80,F1@0.9,Prec@0.90,Rec@0.90,Scores (test),Scores (train),#Samples,#Act (train),#Train actives@0.7,#Actives@0.7
0,JNK3,0.964708,0.864500,0.105900,0.056060,0.954545,0.833703,0.813853,0.854545,0.840295,...,0.404545,0.350746,0.979167,0.213636,$0.58 \pm 0.34$,$0.85 \pm 0.13$,50390,703,188,214
1,GSK3,0.983162,0.926989,0.204161,0.113793,0.991774,0.793747,0.695230,0.924794,0.886106,...,0.485311,0.478648,0.985348,0.316099,$0.67 \pm 0.30$,$0.88 \pm 0.11$,52802,2479,527,620
2,DRD2,0.996084,0.914893,0.207329,0.115671,0.998688,0.801087,0.683673,0.967192,0.847943,...,0.486877,0.463074,0.966667,0.304462,$0.71 \pm 0.26$,$0.89 \pm 0.10$,102981,2219,191,214


In [31]:
latex_table = df_latex.to_latex(float_format="{:,.2f}".format, escape=False, index=False).replace("#", r"\#")
with open(os.path.join(divopt_dir, "data/scoring_functions/stats.tex"), "w") as f:
    f.write(latex_table)